In [469]:
import numpy as np
import pandas as pd
import ast
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
import string
from sklearn.metrics.pairwise import cosine_similarity
from joblib import load, dump

In [360]:
movies=pd.read_csv('../Datasets/tmdb_5000_movies.csv')
credits=pd.read_csv('../Datasets/tmdb_5000_credits.csv')

In [361]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [362]:
movies.shape

(4803, 20)

In [363]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [364]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [365]:
movies=movies.merge(credits, on='title')

 **Columns to be used:**<br>
 - genres
 - id
 - keywords
 - title
 - overview
 - cast
 - crew

In [366]:
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew','release_date']]

In [367]:
movies.isnull().sum()

movie_id        0
title           0
overview        3
genres          0
keywords        0
cast            0
crew            0
release_date    1
dtype: int64

In [368]:
movies.dropna(inplace=True)

In [369]:
movies.isnull().sum()

movie_id        0
title           0
overview        0
genres          0
keywords        0
cast            0
crew            0
release_date    0
dtype: int64

In [370]:
movies.duplicated().sum()

np.int64(0)

In [371]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [372]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew,release_date
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",2009-12-10


In [373]:
def extract_genres(obj):
    genres=[]
    for i in ast.literal_eval(obj):
        genres.append(i['name'])
    return genres

In [374]:
movies['genres']=movies.genres.apply(extract_genres)

In [375]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew,release_date
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",2009-12-10


In [376]:
movies['keywords']=movies.keywords.apply(extract_genres)

In [377]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew,release_date
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",2009-12-10
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",2007-05-19
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",2015-10-26
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",2012-07-16
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",2012-03-07


In [378]:
movies['cast'].iloc[0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [379]:
def extract_cast_names(obj):
    counter=0
    cast_names=[]
    for i in ast.literal_eval(obj):
        if counter!=5:
            cast_names.append(i['name'])
            counter+=1
        else:
            break
    return cast_names

In [380]:
movies['cast']=movies.cast.apply(extract_cast_names)

In [381]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew,release_date
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",2009-12-10


In [382]:
def extract_director(obj):
    directors=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            directors.append(i['name'])
            break
    return directors

In [383]:
movies['director']=movies.crew.apply(extract_director)

In [384]:
movies = movies.drop(columns=['crew'])

In [385]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,release_date,director
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",2009-12-10,[James Cameron]


In [386]:
movies['overview']=movies.overview.apply(lambda x: x.split())

In [387]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,release_date,director
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",2009-12-10,[James Cameron]


In [388]:
movies['year'] = movies.release_date.apply(lambda x: x[:4])

In [389]:
movies = movies.drop(columns=['release_date'])

In [390]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,director,year
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron],2009
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski],2007


In [391]:
movies['genres']=movies.genres.apply(lambda x:[i.replace(" ", "") for i in x])
movies['keywords']=movies.keywords.apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast']=movies.cast.apply(lambda x:[i.replace(" ", "") for i in x])
movies['director']=movies.director.apply(lambda x:[i.replace(" ", "") for i in x])


In [392]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['director']

In [393]:
movies['tags']=movies['tags'].apply(lambda x: ' '.join(x))

In [394]:
new_movies=movies[['movie_id','title','tags']]

In [395]:
new_movies.head(3)

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...


In [398]:
def transform_text(text):
    #tokenize text
    text = word_tokenize(text)
    #remove punctuations
    temp = list()
    for x in text:
        if x not in string.punctuation:
            temp.append(x)
    text = temp
    #make it lower case
    temp = list()
    for x in text:
        temp.append(x.lower())
    text = temp
    #removing stopwords
    temp = list()
    for x in text:
        if x not in stopwords.words("english"):
            temp.append(x)
    #stemming
    ps = PorterStemmer()
    temp = list()
    for x in text:
        temp.append(ps.stem(x))
    return ' '.join(temp)
    

In [402]:
new_movies.loc[:,'tags']=new_movies['tags'].apply(transform_text)

In [406]:
new_movies.head(10)

,movie_id,title,tags
0,19995,Avatar,in the 22nd centuri a parapleg marin is dispat...
1,285,Pirates of the Caribbean: At World's End,captain barbossa long believ to be dead ha com...
2,206647,Spectre,a cryptic messag from bond ’ s past send him o...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,john carter is a war-weari former militari cap...
5,559,Spider-Man 3,the seemingli invinc spider-man goe up against...
6,38757,Tangled,when the kingdom 's most wanted-and most charm...
7,99861,Avengers: Age of Ultron,when toni stark tri to jumpstart a dormant pea...
8,767,Harry Potter and the Half-Blood Prince,as harri begin hi sixth year at hogwart he dis...
9,209112,Batman v Superman: Dawn of Justice,fear the action of a god-lik super hero left u...


**Vectorization : bag of words**

In [451]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=3000, stop_words='english')

In [452]:
vectors=cv.fit_transform(new_movies['tags']).toarray()

In [453]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4805, 3000))

In [454]:
vectors.shape

(4805, 3000)

In [455]:
cv.get_feature_names_out()

array(['000', '10', '11', ..., 'zombi', 'zoo', 'zooeydeschanel'],
      shape=(3000,), dtype=object)

**Distance Calculation: Cosine Distance**

In [456]:
similarity=cosine_similarity(vectors)

In [457]:
movies.columns

Index(['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast',
       'director', 'year', 'tags'],
      dtype='object')

In [458]:
def recommend(movie):
    movie_index=new_movies[new_movies['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x: x[1])[1:11]
    recommend_movie_dict = {
        'title':[],
        'year':[],
        'genres':[],
        'director':[],
        'percentage':[]
    }
    for i in movies_list:
        #print(new_movies.iloc[i[0]].title, i[1]*100)
        recommend_movie_dict['title'].append(movies.iloc[i[0]].title)
        recommend_movie_dict['year'].append(movies.iloc[i[0]].year)
        recommend_movie_dict['genres'].append(movies.iloc[i[0]].genres)
        recommend_movie_dict['director'].append(movies.iloc[i[0]].director)
        recommend_movie_dict['percentage'].append(i[1]*100)
    return recommend_movie_dict

In [459]:
d = recommend('Harry Potter and the Half-Blood Prince')

In [460]:
df=pd.DataFrame(d)

In [461]:
df

,title,year,genres,director,percentage
0,Harry Potter and the Order of the Phoenix,2007,"[Adventure, Fantasy, Family, Mystery]",[DavidYates],38.334909
1,Harry Potter and the Goblet of Fire,2005,"[Adventure, Fantasy, Family]",[MikeNewell],34.358228
2,Harry Potter and the Philosopher's Stone,2001,"[Adventure, Fantasy, Family]",[ChrisColumbus],34.207444
3,Harry Potter and the Chamber of Secrets,2002,"[Adventure, Fantasy, Family]",[ChrisColumbus],33.954988
4,Harry Potter and the Prisoner of Azkaban,2004,"[Adventure, Fantasy, Family]",[AlfonsoCuarón],32.444284
5,The Adventurer: The Curse of the Midas Box,2013,"[Fantasy, Adventure, Family]",[JonathanNewman],31.075944
6,The Indian in the Cupboard,1995,"[Adventure, Family, Fantasy]",[FrankOz],29.192018
7,Stardust,2007,"[Adventure, Fantasy, Romance, Family]",[MatthewVaughn],28.887415
8,Inkheart,2008,"[Adventure, Family, Fantasy]",[IainSoftley],28.736848
9,The Secret of Kells,2009,"[Animation, Family, Fantasy]",[TommMoore],28.460499


#### **Extract files**

In [470]:
with open('../Extracted_Files/movies.joblib', 'wb') as file:
    dump(movies, file)

In [471]:
with open('../Extracted_Files/similarity.joblib', 'wb') as file:
    dump(similarity, file)